# Load Data

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

from Assets import jalali

import plotly.graph_objects as go

# Load Database
# -------------------------------------------------------------------------------------------------------------------------------------------

xls = pd.ExcelFile('Database/HydrographData.xlsx')
Data = pd.read_excel(xls, sheet_name='Data')
GeoInfo = pd.read_excel(xls, sheet_name='GeoInfo').drop(['INDEX'], axis=1)

## GeoInfo
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.rstrip())
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.lstrip())
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.replace('ي','ی'))
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.replace('ئ','ی'))
GeoInfo[COLs] = GeoInfo[COLs].apply(lambda x: x.str.replace('ك', 'ک'))

## Data
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
Data[COLs] = Data[COLs].apply(lambda x: x.str.rstrip())
Data[COLs] = Data[COLs].apply(lambda x: x.str.lstrip())
Data[COLs] = Data[COLs].apply(lambda x: x.str.replace('ي','ی'))
Data[COLs] = Data[COLs].apply(lambda x: x.str.replace('ئ','ی'))
Data[COLs] = Data[COLs].apply(lambda x: x.str.replace('ك', 'ک'))


# Load GeoDatabase
# -------------------------------------------------------------------------------------------------------------------------------------------

## Well Points
gdf = gpd.read_file("GeoDatabase/Wells_Selected.geojson").drop(['INDEX'], axis=1)
gdf = gdf.set_crs("EPSG:32640", allow_override=True)
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
gdf[COLs] = gdf[COLs].apply(lambda x: x.str.replace('ي','ی'))
gdf[COLs] = gdf[COLs].apply(lambda x: x.str.replace('ئ','ی'))
gdf[COLs] = gdf[COLs].apply(lambda x: x.str.replace('ك', 'ک'))

## Boundary
mask = gpd.read_file("GeoDatabase/Aquifers_Selected.geojson")
mask = mask.set_crs("EPSG:32640", allow_override=True)
COLs = ['AQ_NAME', 'MA_NAME']
mask[COLs] = mask[COLs].apply(lambda x: x.str.replace('ي','ی'))
mask[COLs] = mask[COLs].apply(lambda x: x.str.replace('ئ','ی'))
mask[COLs] = mask[COLs].apply(lambda x: x.str.replace('ك', 'ک'))


# Convert Date
# -------------------------------------------------------------------------------------------------------------------------------------------

Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)

Data['DATE_CHECK'] = np.where(
    Data["DATE_PERSIAN_RAW"].isna(),
    np.where(
        Data["DATE_GREGORIAN_RAW"].isna(),
        np.NaN,
        "G"
    ),
    "P"  
)

Data['DATE_PERSIAN_RAW'] = Data.apply(
    lambda x: jalali.Gregorian(x["DATE_GREGORIAN_RAW"].date()).persian_string() if x["DATE_CHECK"] == "G" else x["DATE_PERSIAN_RAW"], 
    axis=1
)

Data['DATE_GREGORIAN_RAW'] = Data.apply(
    lambda x: jalali.Persian(x["DATE_PERSIAN_RAW"]).gregorian_string() if x["DATE_CHECK"] == "P" else x["DATE_GREGORIAN_RAW"], 
    axis=1
)

Data["DATE_GREGORIAN_RAW"] = Data["DATE_GREGORIAN_RAW"].apply(pd.to_datetime)


Data.drop(['DATE_CHECK'], axis=1, inplace=True)

Data.sort_values(
    by=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_GREGORIAN_RAW"], 
    inplace=True
)



# Convert To Day 15
# -------------------------------------------------------------------------------------------------------------------------------------------

def convert_to_day_15(data, date_type="persian"):
    
    data = data.reset_index(drop=True)
            
    df = data[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME"]]
    
    if date_type == "gregorian":
        df["DATE_GREGORIAN"] = data.DATE_GREGORIAN_RAW.apply(pd.to_datetime)
        df["DATE_PERSIAN"] = list(
            map(
                lambda i: jalali.Gregorian(i.date()).persian_string(),
                df["DATE_GREGORIAN"]
            )
        )
        
    elif date_type == "persian":
        df["DATE_PERSIAN"] = data.DATE_PERSIAN_RAW
        df["DATE_GREGORIAN"] = list(
            map(
                lambda i: jalali.Persian(i).gregorian_string(),
                df["DATE_PERSIAN"]
            )
        )
        df["DATE_GREGORIAN"] = df["DATE_GREGORIAN"].apply(pd.to_datetime)
        
    else:
        pass
    
    df["VALUE"] = data.WATER_TABLE_RAW 
           
    df["DELTA_DAY"] = df["DATE_GREGORIAN"].diff().dt.days
    
    df["DATE_PERSIAN_NEW"] = list(
        map(
            lambda i: f"{int(i.split('-')[0])}-{int(i.split('-')[1])}-{15}",
            df["DATE_PERSIAN"]
        )
    )
    
    df["DATE_GREGORIAN_NEW"] = list(
        map(
            lambda i: jalali.Persian(i).gregorian_string(),
            df["DATE_PERSIAN_NEW"]
        )
    )
    
    df["DATE_GREGORIAN_NEW"] = df["DATE_GREGORIAN_NEW"].apply(pd.to_datetime)
    
    df["VALUE_NEW"] = df["VALUE"]
    
    A = []
    
    A.append(df["VALUE"][0])
    
    for i in range(1, len(df) - 1):
        if int(df["DATE_PERSIAN"][i].split('-')[2]) >= 15:
            NEW_VALUE = df["VALUE"][i-1] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i-1]).days) / ((df["DATE_GREGORIAN"][i] - df["DATE_GREGORIAN"][i-1]).days)) * (df["VALUE"][i] - df["VALUE"][i-1]))
            A.append(NEW_VALUE)
        else:
            NEW_VALUE = df["VALUE"][i] + ((((df["DATE_GREGORIAN_NEW"][i] - df["DATE_GREGORIAN"][i]).days) / ((df["DATE_GREGORIAN"][i+1] - df["DATE_GREGORIAN"][i]).days)) * (df["VALUE"][i+1] - df["VALUE"][i]))
            A.append(NEW_VALUE)
    
    A.append(df["VALUE"][len(df) - 1])
            
    df["VALUE_NEW"] = A
        
    return df

Data = Data.drop_duplicates(
    subset=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', 'DATE_GREGORIAN_RAW'],
    keep='last'
)

Data.dropna(
    subset=["WATER_TABLE_RAW"],
    inplace=True
)

Data.reset_index(
    drop=True,
    inplace=True
)

wt_date_converted = Data.groupby(["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME"])\
    .apply(convert_to_day_15)\
        .reset_index(drop=True)[["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN", "DATE_PERSIAN_NEW", "DATE_GREGORIAN", "DATE_GREGORIAN_NEW", "VALUE_NEW"]]

wt_date_converted.columns = ["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW", "DATE_PERSIAN", "DATE_GREGORIAN_RAW","DATE_GREGORIAN", "WATER_TABLE"]

Data = Data.merge(
    right=wt_date_converted,
    how="left",
    on=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_PERSIAN_RAW", "DATE_GREGORIAN_RAW"]
)

Data = Data.drop_duplicates(
    subset=['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME', 'DATE_GREGORIAN'],
    keep='last'
)


# Gap Filling
# -------------------------------------------------------------------------------------------------------------------------------------------

def create_date_day15(min, max):
    """[summary]

    Args:
        min ([type]): [description]
        max ([type]): [description]

    Returns:
        [type]: [description]
    """
    result = []
    min_list = list(map(lambda x: int(x), min.split("-")))
    max_list = list(map(lambda x: int(x), max.split("-")))
    for y in range(min_list[0], max_list[0] + 1):
        for m in range(1, 13):
            result.append(f"{y}-{m}-15")

    result = pd.DataFrame(
        {"DATE_PERSIAN" : result}
    )
    result['DATE_GREGORIAN'] = result.apply(
        lambda x: jalali.Persian(x["DATE_PERSIAN"]).gregorian_string(), 
        axis=1
    )
    result["DATE_GREGORIAN"] = result["DATE_GREGORIAN"].apply(pd.to_datetime)
    result = result[result["DATE_GREGORIAN"] >= pd.to_datetime(jalali.Persian(min).gregorian_string())]
    result = result[result["DATE_GREGORIAN"] <= pd.to_datetime(jalali.Persian(max).gregorian_string())]
    result["DATE_GREGORIAN"] = result["DATE_GREGORIAN"].apply(pd.to_datetime)  
    return result

tmp = pd.DataFrame()

for mn in list(Data["MAHDOUDE_NAME"].unique()):
    df_mn = Data[(Data["MAHDOUDE_NAME"] == mn)]
    
    for an in list(df_mn["AQUIFER_NAME"].unique()):
        df_mn_an = df_mn[(df_mn["AQUIFER_NAME"] == an)]
        
        for ln in list(df_mn_an["LOCATION_NAME"].unique()):
            df_mn_an_ln = df_mn_an[(df_mn_an["LOCATION_NAME"] == ln)]
            
            df_mn_an_ln = df_mn_an_ln.reset_index(drop=False)
            
            date = create_date_day15(
                min = df_mn_an_ln.DATE_PERSIAN.min(),
                max = df_mn_an_ln.DATE_PERSIAN.max()
            ).reset_index(drop=False).sort_values(by=["DATE_GREGORIAN"])
            
            df = date.merge(
                df_mn_an_ln,
                how="left",
                on=["DATE_PERSIAN", "DATE_GREGORIAN"]
            )
            
            df["MAHDOUDE_NAME"] = mn
            df["AQUIFER_NAME"] = an
            df["LOCATION_NAME"] = ln
            df["STORAGE_COEFFICIENT_LOCATION"] = df["STORAGE_COEFFICIENT_LOCATION"].unique()[0]
            
            df = df[[
                "MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME",
                "DATE_GREGORIAN", "DATE_PERSIAN",
                "WATER_TABLE", "STORAGE_COEFFICIENT_LOCATION", "THISSEN_LOCATION", "THISSEN_AQUIFER",
                "DATA_STATE", "DATE_GREGORIAN_RAW", "DATE_PERSIAN_RAW", "WATER_TABLE_RAW"	
            ]]

            tmp = pd.concat([tmp, df], axis=0)

Data = tmp.copy().reset_index(drop=True)


Data['DATE_GREGORIAN_RAW'].fillna(Data['DATE_GREGORIAN'], inplace=True)
Data['DATE_PERSIAN_RAW'].fillna(Data['DATE_PERSIAN'], inplace=True)


del tmp

Data[['YEAR', 'MONTH', 'DAY']] = Data['DATE_PERSIAN_RAW'].str.split('-', 2, expand=True)
Data["YEAR"] = Data["YEAR"].str.zfill(4)
Data["MONTH"] = Data["MONTH"].str.zfill(2)
Data["DAY"] = Data["DAY"].str.zfill(2)
Data['DATE_PERSIAN_RAW'] = Data["YEAR"] + "-" + Data["MONTH"] + "-" + Data["DAY"]


Data = Data[
    ["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_GREGORIAN_RAW", "DATE_PERSIAN_RAW", "WATER_TABLE_RAW", "STORAGE_COEFFICIENT_LOCATION", "THISSEN_LOCATION", "THISSEN_AQUIFER", "DATA_STATE"]
]

Data = Data.sort_values(by=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_GREGORIAN_RAW"]).reset_index(drop=True)


In [ ]:
Data

In [ ]:
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 2)
data = Data.copy()
# data = Data[Data["MAHDOUDE_NAME"].isin(["گیسور"])]
# data = data[data["AQUIFER_NAME"].isin(["گیسور"])]
# data = data[data["LOCATION_NAME"].isin(["کهنه عامر"])]
data = data.sort_values(by=["MAHDOUDE_NAME", "AQUIFER_NAME", "LOCATION_NAME", "DATE_GREGORIAN_RAW"]).reset_index(drop=True)

data["DIFF"] = data["WATER_TABLE_RAW"].diff().abs()
data["DIFF_MEAN"] = data["DIFF"].rolling(6).mean()
data["CHECK_METHOD_1"] = data["DIFF"] > data["DIFF_MEAN"]

data.to_csv("dddl.csv")

data["SHIFT_DATE"] = data["DATE_GREGORIAN_RAW"].shift(periods=1)
data["DIFF_DATE"] = (data["DATE_GREGORIAN_RAW"] - data["SHIFT_DATE"]).dt.days.abs()
data["DERIVATIV"] = (data["DIFF"] / data["DIFF_DATE"]) * 100
data["CHECK_METHOD_2"] = data["DERIVATIV"] > 2


data.describe()

# Database

In [2]:
# -----------------------------------------------------------------------------
# CREATE DATABASE
# -----------------------------------------------------------------------------

import sqlite3
PATH_DB_GROUNDWATER_RAW_DATA = '../../Assets/Database/groundwater_raw_data.db'
DB_GROUNDWATER_RAW_DATA = sqlite3.connect(PATH_DB_GROUNDWATER_RAW_DATA, check_same_thread=False)

# -----------------------------------------------------------------------------
# WRITE DATABASE
# -----------------------------------------------------------------------------

Data["WATER_TABLE_MODIFY"] = Data["WATER_TABLE_RAW"].replace({'0':np.nan, 0:np.nan})

Data.to_sql(
    name="GROUNDWATER_DATA",
    con=DB_GROUNDWATER_RAW_DATA,
    if_exists="replace"
)

GeoInfo.to_sql(
    name="GEOINFO_DATA",
    con=DB_GROUNDWATER_RAW_DATA,
    if_exists="replace"
)

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

from Assets import jalali

import plotly.graph_objects as go

# Load GeoDatabase
# -------------------------------------------------------------------------------------------------------------------------------------------

## Well Points
gdf = gpd.read_file("GeoDatabase/Wells_Selected.geojson").drop(['INDEX'], axis=1)
gdf = gdf.set_crs("EPSG:32640", allow_override=True)
COLs = ['MAHDOUDE_NAME', 'AQUIFER_NAME', 'LOCATION_NAME']
gdf[COLs] = gdf[COLs].apply(lambda x: x.str.replace('ي','ی'))
gdf[COLs] = gdf[COLs].apply(lambda x: x.str.replace('ئ','ی'))
gdf[COLs] = gdf[COLs].apply(lambda x: x.str.replace('ك', 'ک'))

## Boundary
mask = gpd.read_file("GeoDatabase/Aquifers_Selected.geojson")
mask = mask.set_crs("EPSG:32640", allow_override=True)
COLs = ['AQ_NAME', 'MA_NAME']
mask[COLs] = mask[COLs].apply(lambda x: x.str.replace('ي','ی'))
mask[COLs] = mask[COLs].apply(lambda x: x.str.replace('ئ','ی'))
mask[COLs] = mask[COLs].apply(lambda x: x.str.replace('ك', 'ک'))

In [ ]:
j_file["features"][0]

In [ ]:
import plotly.express as px
import json

geodf = mask.copy()

j_file = json.loads(geodf.to_json())

for feature in j_file["features"]:
    feature['id'] = feature['properties']['AQ_NAME']

# fig = px.choropleth_mapbox(
#     data_frame=geodf,
#     geojson=j_file,
#     locations='MA_CODE',
#     opacity=0.4,
# )

fig = px.choropleth_mapbox(
    geodf, 
    geojson=j_file,
    locations='AQ_NAME',
    mapbox_style="carto-positron",
    opacity=0.5
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

# fig.add_trace(
#     go.Scattermapbox(
#         lat=data.Y_Decimal,
#         lon=data.X_Decimal,
#         mode='markers',
#         marker=go.scattermapbox.Marker(size=8),
#         text=data["Well_Name"],
#         hoverinfo='text',
#         hovertemplate='<span style="color:white;">%{text}</span><extra></extra>'
#     )
# )

# fig.add_trace(
#     go.Scattermapbox(
#         lat=selected_wells.Y_Decimal,
#         lon=selected_wells.X_Decimal,
#         mode='markers',
#         marker=go.scattermapbox.Marker(
#             size=10,
#             color='green'
#         ),
#         text=selected_wells["Well_Name"],
#         hoverinfo='text',
#         hovertemplate='<b>%{text}</b><extra></extra>'
#     ), 
# )
    
# fig.update_layout(
#     mapbox = {
#         'style': "stamen-terrain",
#         'zoom': 5,
#     },
#     showlegend = False,
#     hovermode='closest',
#     margin = {'l':0, 'r':0, 'b':0, 't':0}
# )

# fig.show()

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

In [ ]:
import plotly.express as px

fig = px.choropleth_mapbox(
    df, 
    geojson=counties,
    locations='fips',
    mapbox_style="carto-positron",
    opacity=0.5
)
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


In [14]:
A = pd.DataFrame(
    {
        "X": [0, 5, 9, 4, 1, 2, 6, 7, 7, 2, 0, 9]
    }
)

A["DIFF"] = A["X"].diff().abs()


A["DIFF_MEAN"] = A["DIFF"].rolling(3, min_periods=1).mean().shift(1)




A

,X,DIFF,DIFF_MEAN
0,0,NaN,NaN
1,5,5.0,NaN
2,9,4.0,5.000000
3,4,5.0,4.500000
4,1,3.0,4.666667
5,2,1.0,4.000000
6,6,4.0,3.000000
7,7,1.0,2.666667
8,7,0.0,2.000000
9,2,5.0,1.666667
